In [73]:
import py2cytoscape
import networkx as nx
import pandas

In [6]:
from py2cytoscape.data.cyrest_client import CyRestClient

In [96]:
def mknode(name, level):
    return name, level
nodes = [
    mknode("Product", "L0"), 
    mknode("Market", "L0"),
    mknode("Regulatory", "L0"),
    mknode("Business", "L0"),
    mknode("System", "L0.5"),
    mknode("User", "L0.5"),
    mknode("Functional", "L1"),
    mknode("Performance", "L1"),
    mknode("Design", "L2"),
    mknode("Implementation", "L2"),
]

def add_edge(n1, n2, label):
    G.add_edge(n1, n2, link=label)

    
G = nx.DiGraph()
for node, level in nodes:
    G.add_node(node, level=level)

    
G = nx.DiGraph()

add_edge("L0", "L0", "Related Requirements")
add_edge("L1", "L1", "Related Requirements")
add_edge("L2", "L2", "Related Requirements")

add_edge("L0", "L0", "Requirement Decomposition")
add_edge("L1", "L1", "Functional Decomposition")
add_edge("L2", "L2", "Design Decomposition")

add_edge("L0", "L1", "Design Input Translation")
add_edge("L1", "L2", "Requirement Implementation")
add_edge("L0", "L2", "Requirement Implementation")



for l in "L0", "L1", "L2":
    add_edge("Bug", l, "Bug Fix")
    add_edge("Complaint", l, "External Request")
    add_edge("Feature Request", l, "External Request")
    add_edge("Audit Observation", l, "External Request")

In [33]:
cy = CyRestClient()
cy.session.delete()
data = py2cytoscape.util.from_networkx(G)

nw = cy.network.create(None, name="test", collection="Test1", data=data)

In [38]:
nw.get_node_table()

,SUID,id,name,selected,shared name
0,552,L1,L1,False,L1
1,553,L0,L0,False,L0
2,554,Complaint,Complaint,False,Complaint
3,555,L2,L2,False,L2
4,556,Bug,Bug,False,Bug
5,557,Audit Observation,Audit Observation,False,Audit Observation
6,558,Feature Request,Feature Request,False,Feature Request


In [97]:
nodes = ["L0", "L1", "L2", "Complaint", "Feature Request", "Audit Observation", "Bug"]
nnodes = len(nodes)
assert set(nodes) == set(G.nodes())

In [98]:
n2i = {n:i for i, n in enumerate(nodes)}

links = [["" for _ in range(nnodes)] for _ in range(nnodes)]
for e1, e2 in G.edges():
    link = G[e1][e2]['link']
    i1 = n2i[e1]
    i2 = n2i[e2]
    links[i2][i1] = link

In [99]:
df = pandas.DataFrame(links[:-4], index=nodes[:-4], columns=nodes)
df.dropna()
df

,L0,L1,L2,Complaint,Feature Request,Audit Observation,Bug
L0,Requirement Decomposition,,,External Request,External Request,External Request,Bug Fix
L1,Design Input Translation,Functional Decomposition,,External Request,External Request,External Request,Bug Fix
L2,Requirement Implementation,Requirement Implementation,Design Decomposition,External Request,External Request,External Request,Bug Fix


In [124]:
G = nx.DiGraph()
L0 = ["Product", "Market","Regulatory","Business", "System", "User"]
L1 = ["Functional", "Performance"]
L2 = ["Design", "Implementation", "Hardware", "Software"]

for l in L0, L1, L2:
    for ll in l:
        G.add_edge(ll, ll, link="Decomposition")
    
for n1 in L0:
    for n2 in L1:
        G.add_edge(n1, n2, link="Design Input Translation")

    for n2 in L2:
        G.add_edge(n1, n2, link="Requirement Implementation")
  
for n1 in L1:
    for n2 in L2:
        G.add_edge(n1, n2, link="Requirement Implementation")
 
import itertools
for n1 in ("Complaint", "Feature Request", "Audit Observation"):
    for n2 in itertools.chain(L0, L1, L2):
        G.add_edge(n1, n2, link="External Request")
        
for n2 in itertools.chain(L0, L1, L2):
    G.add_edge("Bug", n2, link="Bug Fix")
    
nodes = list(itertools.chain(L0, L1, L2, ("Complaint", "Feature Request", "Audit Observation", "Bug")))
nnodes = len(nodes)

n2i = {n:i for i, n in enumerate(nodes)}

links = [["" for _ in range(nnodes)] for _ in range(nnodes)]
for e1, e2 in G.edges():
    link = G[e1][e2]['link']
    i1 = n2i[e1]
    i2 = n2i[e2]
    links[i2][i1] = link

In [126]:
from officelib.xllib import *
xl = Excel()
wb =xl.Workbooks.Add()
cells = wb.Worksheets(1).Cells
cr = cells.Range
c1=cr("B2")
c2 = c1.GetOffset(len(links)-1, len(links[0])-1)
cr(c1,c2).Value2 = links

c3 = c1.GetOffset(-1, 0)
c4 = c3.GetOffset(0, len(links[0])-1)
cr(c3, c4).Value2 = nodes

c5 = c1.GetOffset(0, -1)
c6 = c1.GetOffset(len(links)-1, 0)

cr(c5, c6).Value2 = [(n,) for n in nodes]


In [116]:
df = pandas.DataFrame(links, index=nodes, columns=nodes)
df

,Product,Market,Regulatory,Business,System,User,Functional,Performance,Design,Implementation,Hardware,Software,Complaint,Feature Request,Audit Observation,Bug
Product,Decomposition,,,,,,,,,,,,External Request,External Request,External Request,Bug Fix
Market,,Decomposition,,,,,,,,,,,External Request,External Request,External Request,Bug Fix
Regulatory,,,Decomposition,,,,,,,,,,External Request,External Request,External Request,Bug Fix
Business,,,,Decomposition,,,,,,,,,External Request,External Request,External Request,Bug Fix
System,,,,,Decomposition,,,,,,,,External Request,External Request,External Request,Bug Fix
User,,,,,,Decomposition,,,,,,,External Request,External Request,External Request,Bug Fix
Functional,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,Decomposition,,,,,,External Request,External Request,External Request,Bug Fix
Performance,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,,Decomposition,,,,,External Request,External Request,External Request,Bug Fix
Design,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Decomposition,,,,External Request,External Request,External Request,Bug Fix
Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,,Decomposition,,,External Request,External Request,External Request,Bug Fix
